<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fapplied_numerical_finance/applied_numerical_finance/estimating_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Numerical Finance

* Group: Dominik Meyer, Mikhail Borovkov, Brage Bakken, Emanuele Chiarini, Enrico Giannelli
* Assignment: 10 Finite difference approach for the Greeks

In [ ]:
# import required packages
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import norm
from scipy.optimize import minimize
from typing import Optional, List, Tuple
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *
from plotly.subplots import make_subplots

In [ ]:
# define parameters
S = 100
K = 100
T = 1
r = 0.01
q=0
sigma = 0.30
n=10000
prices = np.arange(1, 200,1)

In [ ]:
def generate_path(S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, seed: Optional[int]=False, n: int=1) -> np.ndarray:
  """
  Function that generates the price of a European option.

  Parameters
  ----------
  S : float
      The current stock price.
  K : float
      The strike price of the option.
  T : float
      The time to expiration of the option.
  r : float
      The risk-free interest rate.
  q : float
      The dividend yield.
  sigma : float
      The volatility of the stock.
  direction : str
      The option direction, should be 'call' or 'put'.
  seed : Optional[int]
      Optional seed for the random number generator. Default is False
  n : int
      Number of prices to generate to run.

  Returns
  -------
  np.ndarray
      An array of option prices, one for each simulation.
  """
  if seed!= False:
    np.random.seed(seed) # Set the seed for the random number generator if specified
  Z = np.random.normal(size=n) # Generate random numbers from a normal distribution
  fin_S = S*np.exp((r-q-(sigma**2)/2)*T + sigma*np.sqrt(T)*Z) # Calculate the final stock prices
  if direction=='call':
    return np.exp(-r*T)*np.max((fin_S - K,np.zeros(n)), axis=0) # Calculate the call option prices
  elif direction=='put':
    return np.exp(-r*T)*np.max((K - fin_S,np.zeros(n)), axis=0) # Calculate the put option prices

In [ ]:
def expected_price_option(S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str) -> float:
    """
    Function that computes the expected price of an option using the Black Sholes formula.

    Args:
        S : float
            The current stock price.
        K : float
            The strike price of the option.
        T : float
            The time to expiration of the option.
        r : float
            The risk-free interest rate.
        q : float
            The dividend yield.
        sigma : float
            The volatility of the stock.
        direction : str
            The option direction, should be 'call' or 'put'.

    Returns:
        float: The expected price of the option.
    """
    # Calculate the d1 and d2 values, which are used in the Black-Scholes formula
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    # Calculate the call and put prices using the Black-Scholes formula
    if direction=='call':
        # European call price
        c = S*np.exp(-q*T)*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        return c
    elif direction=="put":
        # European put price
        p = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*np.exp(-q*T)*norm.cdf(-d1, 0, 1)
        return p


In [ ]:
def compute_greek(S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str) -> float:

    """
    Computes the value of a specified "greek" for a given option using the theoretical formulas.

    Parameters:
    -----------
    S : float
        The underlying asset price.
    K : float
        The option strike price.
    T : float
        The time to expiration of the option.
    r : float
        The risk-free interest rate.
    q : float
        The dividend rate.
    sigma : float
        The volatility of the underlying asset.
    direction : str
        The option direction, either "call" or "put".
    greek : str
        The greek to compute, one of "delta", "gamma", "theta", "vega", or "rho".

    Returns:
    --------
    float
        The value of the specified greek for the given option.
    """
    # define general pricing parameters
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    # if the greek to compute is "delta", compute the delta
    if greek == "delta":
      # if the option is a call, return the call delta
      if direction == "call":
        return np.exp(-q*T)*norm.cdf(d1, 0, 1)
      # if the option is a put, return the put delta
      elif direction == "put":
        return -np.exp(-q*T)*norm.cdf(-d1, 0, 1)

    # if the greek to compute is "gamma", compute the gamma
    elif greek == "gamma":
      return np.exp(-q*T)*norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))

    # if the greek to compute is "theta", compute the theta
    elif greek == "theta":
      # if the option is a call, return the call theta
      if direction == "call":
        return (-S*np.exp(-q*T)*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1) + q*S*np.exp(-q*T)*norm.cdf(d1, 0, 1))/365
      # if the option is a put, return the put theta
      elif direction == "put":
        return (-S*np.exp(-q*T)*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - q*S*np.exp(-q*T)*norm.cdf(-d1, 0, 1))/365

    # if the greek to compute is "vega", compute the vega
    elif greek == "vega":
      return S*np.exp(-q*T)*norm.pdf(d1, 0, 1)*np.sqrt(T)*0.01

    # if the greek to compute is "rho", compute the rho
    elif greek == "rho":
      # if the option is a call, return the call rho
      if direction == "call":
        return K*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)*0.01
      # if the option is a put, return the put rho
      elif direction == "put":
        return -K*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)*0.01

In [ ]:
def estimate_greeks(S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str, estimation_type: str = "closed_form", 
estimation_method: str = None, path_seed: float = False, n: int =10000, h: float=0.01) -> float:

    """
    This function either computes greek via closed form formulas or estimates them through finite differences.

    Parameters
    ----------
    S : float
        The current stock price.
    K : float
        The strike price of the option.
    T : float
        The time to expiration of the option, in years.
    r : float
        The annualized risk-free interest rate, as a decimal.
    q : float
        The annualized continuous dividend yield, as a decimal.
    sigma : float
        The annualized volatility of the underlying stock, as a decimal.
    direction : str
        The direction of the option, either "call" or "put".
    greek : str
        The greek to estimate, either "delta", "vega", "theta", "rho", or "gamma".
    estimation_type : str, optional
        The type of estimation to use, either "mc" (Monte Carlo), "closed_form", or "exact_fd" (exact finite difference).
        The default is "closed_form".
    estimation_method : str, optional
        The method to use for finite difference estimation, either "forward_difference" or "central_difference".
        This parameter is ignored if estimation_type is not "exact_fd" or "mc".
    path_seed : float, optional
        A seed for the random number generator used to generate sequences of stock prices when using Monte Carlo estimation. Default is False
    n : int
        The number of price paths to be generated for Monte Carlo simulation. Default is 10000
    h : float
        The increment used in the finite difference estimator. Default is 0.01
    
    Returns:
    --------
    float
        the computed/estimated value of the greek, depending on the values of the input arguments
    """
        
    # if the estimation type is "closed_form", compute the greek using the closed-form formula
    if estimation_type=="closed_form":
        return compute_greek(S, K, T, r, q, sigma, direction, greek)

    # set the increment for the finite difference estimator
    div=h

    # create dictionaries of the input parameters
    parameters={"S":S,"K":K,"T":T, "r":r, "q":q, "sigma":sigma}
    parameters_2=parameters.copy()
    parameters_3=parameters.copy()

    # set the multiplicative constant for the greek estimate
    multiplicative_constant=1

    # set the parameter that will be varied to compute the derivative
    if greek == "delta":
        par="S"

    elif greek == "vega":
        par="sigma"
        multiplicative_constant=0.01

    elif greek == "theta":
        par="T"
        multiplicative_constant=-1/365
    
    elif greek == "rho":
        par="r"
        multiplicative_constant=0.01

    #the estimation of gamma, being a second order derivative, is done separately from the others
    elif greek == "gamma":
        par="S"
        if estimation_method=='forward_difference':
            parameters["S"]=parameters["S"]+h
            parameters_2["S"]=parameters_2["S"]+h
    # depending on the method of finite difference estimation, set the parameter values for the three cases
    if estimation_method in ['forward_difference',  "central_difference"]:
        parameters[par]=parameters[par]+h
    if estimation_method=="central_difference": 
        parameters_3[par]=parameters_3[par]-h
        div=2*h

    # if the estimation type is "exact_fd", compute the option price for the two/three cases using the black sholes formula
    if estimation_type=="exact_fd":
            price_1=expected_price_option(S=parameters["S"], K=parameters["K"], T=parameters["T"], r=parameters["r"], q=parameters["q"], sigma=parameters["sigma"], 
            direction=direction)
            price_3=expected_price_option(S=parameters_3["S"], K=parameters_3["K"], T=parameters_3["T"], r=parameters_3["r"], q=parameters_3["q"], sigma=parameters_3["sigma"], 
            direction=direction)
            if greek=="gamma":
                price_2=expected_price_option(S=parameters_2["S"], K=parameters_2["K"], T=parameters_2["T"], r=parameters_2["r"], q=parameters_2["q"], sigma=parameters_2["sigma"], 
                direction=direction)
                div=h**2
                result = (price_1 - 2*price_2+ price_3)/div
            else:
                result=(price_1-price_3)/div
    
    # if the estimation type is "mc", use Monte Carlo simulation to generate option prices for the two/three cases
    elif estimation_type=="mc":
            prices_1 = generate_path(S=parameters["S"], K=parameters["K"], T=parameters["T"], r=parameters["r"], q=parameters["q"], sigma=parameters["sigma"], 
            direction=direction, seed=path_seed, n=n)  
            prices_3 = generate_path(S=parameters_3["S"], K=parameters_3["K"], T=parameters_3["T"], r=parameters_3["r"], q=parameters_3["q"], sigma=parameters_3["sigma"], 
            direction=direction, seed=path_seed, n=n)  
            if greek=="gamma":
                prices_2 = generate_path(S=parameters_2["S"], K=parameters_2["K"], T=parameters_2["T"], r=parameters_2["r"], q=parameters_2["q"], sigma=parameters_2["sigma"], 
                direction=direction, seed=path_seed, n=n)  
                div=h**2
                result = (np.mean(prices_1) - 2*np.mean(prices_2)+ np.mean(prices_3))/div
            else:
                result = (np.mean(prices_1) - np.mean(prices_3))/div
                
    return result*multiplicative_constant

In [ ]:
def variance_estimation(n_var: int, S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str, 
estimation_method: str, path_seed: float = False, n: int = 1000, h: float = 0.01):  
    """
    Estimates the variance of the First difference approach using the `estimate_greeks` function.
    
    Parameters
    ----------
    n_var : int
        The number of first difference estimates to perform.
    S : float
        The current stock price.
    K : float
        The strike price of the option.
    T : float
        The time to expiration of the option, in years.
    r : float
        The annualized risk-free interest rate, as a decimal.
    q : float
        The annualized continuous dividend yield, as a decimal.
    sigma : float
        The annualized volatility of the underlying stock, as a decimal.
    direction : str
        The direction of the option, either "call" or "put".
    greek : str
        The greek to estimate, either "delta", "vega", "theta", "rho", or "gamma".
    estimation_method : str
        The method to use for finite difference estimation, either "forward_difference" or "central_difference".
    path_seed : float, optional
        A seed for the random number generator used to generate sequences of stock prices when using Monte Carlo estimation
    n : int
        The number of price paths to be generated for Monte Carlo simulation. Default is 10000
    h : float
    The increment used in the finite difference estimator. Default is 0.01
            
    Returns:
    --------
    float
       the variance of the first difference estimates, as a scalar
    """

    res = np.zeros([n_var,3])

    #if no seed specified create vector of Falses
    if isinstance(path_seed, np.bool_):
      path_seed=np.repeat(False, n_var)

    # use the `estimate_greeks` function to compute n_var first difference estimates of the greek
    for i in range(n_var):
      res[i,:]=estimate_greeks(S, K, T, r, q, sigma, direction, greek, "mc", estimation_method, path_seed[i], n, h)     
    # return the variance of the first difference estimates
    return np.var(res[:,0])


In [ ]:
#Variable used for plotting purposes
estimation_methods={"central_difference":"CDE","forward_difference":"FDE"}

def plot_error(prices: List[float], K: float, T: float, r: float, q: float, sigma: float, greek: str, estimation_method: str) -> go.Figure:
    """
    Plots the error for the specified greek.

    Parameters
    ----------
    prices : array-like
        The stock prices to check the error against.
    K : float
        The strike price.
    T : float
        The time to expiration.
    r : float
        The risk-free interest rate.
    q : float
        The dividend yield.
    sigma : float
        The volatility.
    greek : str
        The greek to plot the error for.
    estimation_method : str
        The method to use for finite difference estimation, either "forward_difference" or "central_difference".

    Returns
    -------
    fig : plotly.graph_objects.Figure
    The plotly figure object plotting the error.

    """
    # Compute the closed-form estimate of the greek
    final_results = pd.DataFrame()
    final_results['closed_form'] = estimate_greeks(S=prices, K=K, T=T, r=r, q=q, sigma=sigma, direction='call', greek=greek, estimation_type='closed_form', path_seed=False)

    # Define the increments to use for the finite difference approximation
    increments = [0.001,0.1, 0.5, 1, 1.5, 2.0, 2.5,3]

    # Create an empty list to store the data for the plot
    data = []

    # Loop over the increments
    for h in increments:
        # Compute the finite difference approximation of the greek
        finite_difference = estimate_greeks(S=prices, K=K, T=T, r=r, q=q, sigma=sigma, direction='call',  greek=greek, estimation_type='exact_fd', estimation_method=estimation_method, path_seed=False, h=h)

        # Compute the error between the closed-form and finite difference estimates
        estimation_error = final_results['closed_form'] - finite_difference

        # Add the error to the final results dataframe
        final_results['error (h={})'.format(str(h))] = estimation_error

        # Create a trace for the error and add it to the data for the plot
        trace = go.Scatter(x=prices, y=final_results["error (h={})".format(str(h))], mode='lines', line_color='tomato', name="error (h={})".format(str(h)))
        data.append(trace)
    
    layout = go.Layout(template="plotly_dark", title_x=0.5, showlegend=True, legend_tracegroupgap=280, title='Error for greek {} estimated by {}'.format(greek,estimation_methods[estimation_method], width=800,
    height=600),
    xaxis={'showgrid': True, 'gridcolor': 'grey', 'ticks': 'outside', 'tickwidth': 1, 'tickcolor': 'grey', 'ticklen': 5, 'linecolor': 'grey','title': 'Stock price'}, 
    yaxis={'showgrid': True, 'gridcolor': 'grey', 'ticks': 'outside' , 'tickwidth': 1, 'tickcolor': 'grey', 'ticklen': 5, 'linecolor': 'grey', 'title': 'Error'})

    # Create the figure using the data and layout
    fig = go.Figure(data=data, layout=layout )

    # Return the plot
    return fig

In [ ]:
def create_plot_simulations(data: dict, title: str, ma: bool = False, half_plot: bool= False) -> go.Figure:
  """
  Plots the greek estimations of the Monte Carlo simulation and the closed form solution.

  Parameters
  ----------
  data : dict
      A dictionary containing the x and y values to plot, as well as the x and y axis titles plus the boundaries of the y axis.
  title : str
      The title of the plot.
  ma : int, optional
      The window size to use for the moving average of the Monte Carlo estimations. Default is False which doesn't plot the moving average.
  half_plot : bool, optional
      Determines whether to half the horizontal size of the plot. Default is False
  Returns
  -------
  fig : plotly.graph_objects.Figure
      The plotly figure object plotting estimations.
  """
  # create figure object
  fig = go.Figure()
  
  # add trace for Monte Carlo estimate
  fig.add_trace(go.Scatter(
    x=data['x'], 
    y=data['y'], 
    mode='lines', 
    line_color='slateblue', 
    name="monte carlo", 
    legendgroup='1'
  ))

  # add trace for closed form estimate
  fig.add_trace(go.Scatter(
    x=data['x'], 
    y=data['y_closed'], 
    mode='lines', 
    line_color='yellow', 
    name="closed form", 
    legendgroup='1'
  ))

  # add trace for moving average if window size is provided
  if ma:
    window_size = ma
    moving_avg_1 = np.convolve(data['y'], np.ones(window_size)/window_size, mode='valid')
    fig.add_trace(go.Scatter(
      x=data['x'][int(window_size/2):-int(window_size/2)], 
      y=moving_avg_1, 
      mode='lines', 
      line_color='tomato', 
      name="moving average", 
      legendgroup='1'
    ))

  # set the plot title and axes labels
  fig.update_layout(
    title_text=title, 
    template="plotly_dark", 
    xaxis_title=data["x_axis"], 
    yaxis_title=data["y_axis"]
  )

  #if the bounds for the y axis are provided set them
  if data["y_max"] is not None:
    fig.update_layout(yaxis_range=[data["y_min"],data["y_max"]])

  fig.update_layout(
  width=1333,
  height=500)


  #if half_plot is True cut the width of the plot in half
  if half_plot:
    fig.update_layout(
    width=500,
    height=400)

  return fig

In [ ]:
def plot_greek_mc(S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str,path_seed: Optional[List[int]]=False, 
                  n_try: Optional[List[int]] = None, h_try : Optional[List[int]] = None, n: int =10000, h: float =0.01, ma: bool =False,
                  same_axis: bool=False, half_plot: bool=False) -> Tuple[go.Figure, go.Figure]:
  """
  Plots the greek estimations of the Monte Carlo simulation and the closed form solution.

  Parameters
  ----------
  S : float
      The spot price of the underlying asset.
  K : float
      The strike price of the option.
  T : float
      The time to expiration of the option, in years.
  r : float
      The annualized risk-free interest rate.
  q : float
      The annualized dividend yield.
  sigma : float
      The annualized volatility of the underlying asset.
  direction : str
      The option direction, either 'call' or 'put'.
  greek : str
      The greek value to estimate, either 'delta', 'vega'. 'rho', 'theta', or 'gamma'.
  path_seed : array-like, optional
      The seed value for the random number generator used to generate Monte Carlo paths. The size of this should be equl to that of n_try or h_try.
      Default is False.
  n_try : array-like, optional
      The values of n to use for the Monte Carlo simulation. Either this or h_try needs to be specified.
  h_try : array-like, optional
      The values of h to use for the Monte Carlo simulation. Either this or n_try needs to be specified.
  n : int, optional
      The number of Monte Carlo simulations to run, defaults to 10000.
  h : float, optional
      The step size to use for the Monte Carlo simulation, defaults to 0.01.
  ma : int, optional
      The window size to use for the moving average of the Monte Carlo estimations. Default is False.
  same_axis : bool, optional
      Determines whether the y axis is the same for the CDE and FDE plot. Default is False.
  half_plot : bool, optional
      Determines whether to half the horizontal size of the plot. Default is False.

  Returns
  -------
  plot_1 : plotly.graph_objects.Figure
      The plotly figure object for the FDE estimation.
  plot_2 : plotly.graph_objects.Figure
      The plotly figure object for the CDE estimation.
  """
  # create an empty dataframe to store the simulation results
  sim_results = pd.DataFrame()

  # if n_try is not None, we will run the simulations with different values of n
  if n_try is not None:
    #if no seed specified create vector of Falses
    if isinstance(path_seed,bool):
      path_seed=np.repeat(False, len(n_try))
    # estimate the greeks using the Monte Carlo forward difference method with different values of n and store them in the dataframe
    sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek, estimation_type="mc", 
                                                      estimation_method='forward_difference', path_seed=j, n=i, h=h) for i,j in zip(n_try,path_seed)]
    # estimate the greeks using the Monte Carlo central difference method with different values of n and store them in the dataframe
    sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek, estimation_type="mc", 
                                                      estimation_method='central_difference', path_seed=j, n=i, h=h) for i,j in zip(n_try,path_seed)]
    # estimate the greeks using the closed form solution with different values of n and store them in the dataframe
    sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek, estimation_type='closed_form',
                                                  n=i, h=h) for i in n_try]

    # create a dictionary with the data to plot the CDE estimate over simulations
    data_1 = {
      'x': n_try,  # values of n to use on the x-axis
      'y_closed': sim_results['closed_form'],  # values of the closed form solution to use on the y-axis
      'y': sim_results['monte_carlo_fde'],  # values of the Monte Carlo CDE estimate to use on the y-axis
      'x_axis': "n",  # x-axis title
      'y_axis': greek,  # y-axis title
      'y_max': None,
      'y_min': None
    }

    #if same_axis is set to True set the CDE axis equal to the FDE's ones 
    if same_axis:
      y_max=np.max(sim_results['monte_carlo_fde'])
      y_min=np.min(sim_results['monte_carlo_fde'])
    else:
      y_max=None
      y_min=None

    # create the plot using the create_plot_simulations function
    plot_1 = create_plot_simulations(data_1, "FDE estimate over simulations", ma, half_plot)

    # create a dictionary with the data to plot the FDE estimate over simulations
    data_2 = {
      'x': n_try,  # values of n to use on the x-axis
      'y_closed': sim_results['closed_form'],  # values of the closed form solution to use on the y-axis
      'y': sim_results['monte_carlo_cde'],  # values of the Monte Carlo FDE estimate to use on the y-axis
      'x_axis': "n",  # x-axis title
      'y_axis': greek,  # y-axis title
      'y_max': y_max,
      'y_min': y_min
    }
    
    # create the plot using the create_plot_simulations function
    plot_2 = create_plot_simulations(data_2, "CDE estimate over simulations",ma,half_plot)

  # if h_try is not None, we will run the simulations with different values of h
  if h_try is not None:
    #if no seed specified create vector of Falses
    if isinstance(path_seed,bool):
      path_seed=np.repeat(False, len(h_try))
    # estimate the greeks using the Monte Carlo forward difference method with different values of h and store them in the dataframe
    sim_results['monte_carlo_fde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek, estimation_type="mc",
                                                      estimation_method='forward_difference', path_seed=j, n=n, h=i) for i,j in zip(h_try,path_seed)]
    # estimate the greeks using the Monte Carlo central difference method with different values of h and store them in the dataframe
    sim_results['monte_carlo_cde'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek, estimation_type="mc",
                                                      estimation_method='central_difference', path_seed=j, n=n, h=i) for i,j in zip(h_try,path_seed)]
    # estimate the greeks using the closed form solution with different values of h and store them in the dataframe
    sim_results['closed_form'] = [estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek, 
                                                  estimation_type='closed_form', n=n, h=i) for i in h_try]
    
    # create a dictionary with the data to plot the FDE estimate over h
    data_1 = {
      'x': h_try,  # values of h to use on the x-axis
      'y_closed': sim_results['closed_form'],  # values of the closed form solution to use on the y-axis
      'y': sim_results['monte_carlo_fde'],  # values of the Monte Carlo FDE estimate to use on the y-axis
      'x_axis': "h",  # x-axis title
      'y_axis': greek,  # y-axis title
      'y_max': None,
      'y_min': None
      }

    #if same_axis is set to True set the CDE axis equal to the FDE's ones 
    if same_axis == True:
      y_max=np.max(sim_results['monte_carlo_fde'])
      y_min=np.min(sim_results['monte_carlo_fde'])
    else:
      y_max= None
      y_min=None
    
    # create the plot using the create_plot_simulations function
    plot_1 = create_plot_simulations(data_1, "FDE estimate over h",ma,half_plot)

    # create a dictionary with the data to plot the CDE estimate over h
    data_2 = {
      'x': h_try,  # values of n to use on the x-axis
      'y_closed': sim_results['closed_form'],  # values of the closed form solution to use on the y-axis
      'y': sim_results['monte_carlo_cde'],  # values of the Monte Carlo CDE estimate to use on the y-axis
      'x_axis': "h",  # x-axis title
      'y_axis': greek,  # y-axis title
      'y_max': y_max,
      'y_min': y_min
    }

    # create the plot using the create_plot_simulations function
    plot_2 = create_plot_simulations(data_2, "CDE estimate over h",ma, half_plot)   
  return plot_1, plot_2


In [ ]:
def create_plot_variance(data: dict, title: str, half_plot : bool =False) -> go.Figure:
  """
  Plots the variance of the Monte Carlo estimations.

  Parameters
  ----------
  data : dict
      A dictionary containing the x and y values to plot, as well as the x axis title.
  title : str
      The title of the plot.
  half_plot : bool, optional
      Determines whether to half the horizontal size of the plot. Default is False

  Returns
  -------
  fig : plotly.graph_objects.Figure
      The plotly figure object of the variance plot.
  """
  
  fig = go.Figure()

  # add trace to the plot
  fig.add_trace(go.Scatter(
    x=data['x'], # x-values
    y=data['y'], # y-values
    mode='lines', # use lines to connect the data points
    line_color='orange', # color of the line
    name="variance", # name of the trace
    legendgroup='1' # group with other traces
  ))

  # set the plot title
  fig.update_layout(
    title_text=title, # title of the plot
    template="plotly_dark", # plotly template to use
    xaxis_title=data["x_axis"], # x-axis title
    yaxis_title="variance" # y-axis title
  )

  #if half_plot is True cut the width of the plot in half
  if half_plot:
    fig.update_layout(
    width=500,
    height=400)

  return fig

In [ ]:
def plot_variance_greeks(n_var: int, S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str,
path_seed: Optional[List[int]]=False, n_try: Optional[List[int]] = None, h_try: Optional[List[int]] = None, n: int = 10000, h: float = 0.01, 
half_plot: bool = False) -> Tuple[go.Figure, go.Figure]:  
  """
  Plots the variance of the Greeks' Monte Carlo simulation.

  Parameters
  ----------
  n_var : int
      The number of first difference estimates to perform.
  S : float
      The spot price of the underlying asset.
  K : float
      The strike price of the option.
  T : float
      The time to expiration of the option, in years.
  r : float
      The annualized risk-free interest rate.
  q : float
      The annualized dividend yield.
  sigma : float
      The annualized volatility of the underlying asset.
  direction : str
      The option direction, either 'call' or 'put'.
  greek : str
      The greek value to estimate, either 'delta', 'vega'. 'rho', 'theta', or 'gamma'.
  path_seed : array-like, optional
      The seed value for the random number generator used to generate Monte Carlo paths. The size of this should be equal to that of n_try or h_try. 
      Default is False.
  n_try : array-like, optional
      The values of n to use for the Monte Carlo simulation. Either this or h_try needs to be specified.
  h_try : array-like, optional
      The values of h to use for the Monte Carlo simulation. Either this or h_try needs to be specified.
  n : int, optional
      The number of Monte Carlo simulations to run, defaults to 10000.
  h : float, optional
      The step size to use for the Monte Carlo simulation, defaults to 0.01.
  half_plot : bool, optional
      Determines whether to half the horizontal size of the plot. Default is False

  Returns
  -------
  plot_1 : plotly.graph_objects.Figure
      The plotly figure object for the CDE plot.
  plot_2 : plotly.graph_objects.Figure
      The plotly figure object for the FDE plot.
  """

  # if n_try is not None, we will run the simulations with different values of n
  if n_try is not None:
    #if no seed specified create vector of Falses
    if isinstance(path_seed,bool):
      path_seed=np.repeat(False, len(n_try)).all
  
    # estimate the variance of the the Monte Carlo CDE method with different values of n and store them 
    sim_results_var=[variance_estimation(n_var, S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction,  greek=greek, 
                                         estimation_method='central_difference', path_seed=j, n=i, h=h) for i,j in zip(n_try,path_seed) ]

    data_1 = {
      'x': n_try,  # values of n to use on the x-axis
      'y': sim_results_var,  # values of the Monte Carlo CDE estimate to use on the y-axis
      'x_axis': "n",  # x-axis title
    }
    # create the plot using the create_plot_variance function
    fig_cde = create_plot_variance(data_1, "CDE variance estimate over n", half_plot)


    # estimate the variance of the the Monte Carlo FDE method with different values of n and store them 
    sim_results_var=[variance_estimation(n_var, S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction,  greek=greek, 
                                         estimation_method='forward_difference', path_seed=j, n=i, h=h) for i,j in zip(n_try,path_seed) ]

    data_2 = {
      'x': n_try,  # values of n to use on the x-axis
      'y': sim_results_var,   # values of the Monte Carlo FDE estimate to use on the y-axis
      'x_axis': "n",  # x-axis title
    }

    # create the plot using the create_plot_variance function
    fig_fde = create_plot_variance(data_2, "FDE variance estimate over n", half_plot)

  # if h_try is not None, we will run the simulations with different values of n
  elif h_try is not None:
    #if no seed specified create vector of Falses
    if isinstance(path_seed,bool):
      path_seed=np.repeat(False, len(h_try))
    # estimate the variance of the the Monte Carlo CDE method with different values of h and store them 
    sim_results_var=[variance_estimation(n_var, S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction,  greek=greek, 
                                         estimation_method='central_difference', path_seed=j, n=n, h=i) for i,j in zip(h_try,path_seed)]
    
    data_1 = {
      'x': h_try, # values of h to use on the x-axis
      'y': sim_results_var,   # values of the Monte Carlo CDE estimate to use on the y-axis
      'x_axis': "h",  # x-axis title
    }
    # create the plot using the create_plot_variance function
    fig_cde = create_plot_variance(data_1, "CDE variance estimate over h", half_plot)

    # estimate the variance of the the Monte Carlo FDE method with different values of h and store them 
    sim_results_var=[variance_estimation(n_var, S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction,  greek=greek, 
                                         estimation_method='forward_difference', path_seed=j, n=n, h=i) for i,j in zip(h_try,path_seed) ]
    data_2 = {
      'x': h_try,  # values of h to use on the x-axis
      'y': sim_results_var,   # values of the Monte Carlo FDE estimate to use on the y-axis
      'x_axis': "h",  # x-axis title
    }
    # create the plot using the create_plot_variance function
    fig_fde = create_plot_variance(data_2, "FDE variance estimate over h", half_plot)

  return fig_cde, fig_fde



# Theoretical biases

In [ ]:
fig_1=plot_error(prices, K, T, r, q, sigma, "delta", "central_difference")

fig_1.show()

<ipython-input-4-f128d990abf8>:25: RuntimeWarning: divide by zero encountered in log
  d1 = (np.log(S/K) + (r - q + sigma**2/2)*T)/(sigma*np.sqrt(T))
<ipython-input-4-f128d990abf8>:25: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S/K) + (r - q + sigma**2/2)*T)/(sigma*np.sqrt(T))


# The role of h

In [ ]:
p_1,p_2=plot_greek_mc(S, K, T, r, q, sigma,'call','delta',False , h_try=np.linspace(0.4,20,400), ma=10,n=10000)

p_1.show()

In [ ]:
fig_1,fig_2=plot_variance_greeks(200,S,K,T,r,q,sigma, "call", "delta", False, h_try=np.linspace(0.4,20,400), n=10000)

fig_2.show()

# The role of n

In [ ]:
p_1,p_2=plot_greek_mc(S, K, T, r, q, sigma,'put','delta', np.arange(1,1000), n_try=np.arange(10,10000,10), ma=False,half_plot=True)

p_2.show()

In [ ]:
fig_1,fig_2=plot_variance_greeks(200,S,K,T,r,q,sigma, "put", "delta", np.reshape(np.arange(1,200*999+1),(999,200)),n_try=np.arange(10,10000,100) ,half_plot=True)

fig_1.show()

# CDE VS FDE

In [ ]:
p_1,p_2=plot_greek_mc(S, K, T, r, q, sigma,'call','vega', False, h_try=np.linspace(0.2,0.3,400), ma=10, n=10000, same_axis=True)

p_1.show()
p_2.show()

# Same seed

In [ ]:
p_1,p_2=plot_greek_mc(S, K, T, r, q, sigma,'call','gamma', False, h_try=np.linspace(1,1.2,400), n=1000, same_axis=False)

p_2.show()

In [ ]:
p_1,p_2=plot_greek_mc(S, K, T, r, q, sigma,'call','gamma', np.arange(1,401), h_try=np.linspace(1,1.2,400), n=10000, same_axis=False)

p_2.show()

In [ ]:
def bias(x: float, S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str, 
         estimation_method: str = None, path_seed:  Optional[List[int]]=False, n: int =10000, run: int=500, verbose: bool = False) -> float:
    """
    Calculates the mse of an option price estimate. 
    
    Parameters
    ----------
    x : float
        The value of h to use in the finite difference method.
    S : float
        The underlying asset price.
    K : float
        The strike price of the option.
    T : float
        The time to expiration of the option.
    r : float
        The risk-free interest rate.
    q : float
        The dividend yield of the underlying asset.
    sigma : float
        The volatility of the underlying asset.
    direction : str
        The direction of the option ("call" or "put").
    greek : str
        The greek to estimate ("delta", "gamma", "theta", or "vega").
    estimation_method : str, optional
        The method to use to estimate the option price in the finite difference method. Either "forward_difference" or "central_difference"
    path_seed : array-like, optional
        The seeds to use for the random number generator in the simulation. Default is False, else an array of size 'run' needs to be provided.
    n : int, optional
        The number of samples to use in the simulation. Default is 10000
    run : int, optional
        The number of estimates on which to compute the MSE. Default is 500.
    verbose : bool, optional
        A flag indicating whether to print intermediate results.
        
    Returns
    -------
    float
        The bias of the option price estimate, calculated as the mean squared error over 500 samples.
    """
    
    # Initialize the mean squared error to 0
    mse = 0

    #if no seed specified create vector of Falses
    if isinstance(path_seed,bool):
      path_seed=np.repeat(False, run)

    # Loop over 500 samples
    for i in range(1, run):
        # Get the closed-form solution for the option price
        closed_form = estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek,
                                      estimation_type='closed_form')
        # Get the finite difference estimate for the option price
        finite_difference = estimate_greeks(S=S, K=K, T=T, r=r, q=q, sigma=sigma, direction=direction, greek=greek,
                                            estimation_type='mc', estimation_method=estimation_method, path_seed=path_seed[i],
                                            h=x, n=n)
        # Add the squared error for this sample to the mean squared error
        mse+=np.power(closed_form-finite_difference,2)
    mse=mse/run
    #If verbose is True print intermediate values
    if verbose==True:
      print("The current value of h is ",x, " the error is: ",mse)
    return mse

In [ ]:
def optimizer(h0: float, S: float, K: float, T: float, r: float, q: float, sigma: float, direction: str, greek: str, 
              estimation_method: str = None, path_seed:  Optional[List[int]]=False, n: int =10000, run: int =500, verbose : bool = False) -> float:
    """
    Optimizes the bias of an option price (estimated via the first difference approach) with respect to h using the COBYLA method.
        
    Parameters
    ----------
    h0 : float
        The initial value of h to use in the optimization.
    S : float
        The underlying asset price.
    K : float
        The strike price of the option.
    T : float
        The time to expiration of the option.
    r : float
        The risk-free interest rate.
    q : float
        The dividend yield of the underlying asset.
    sigma : float
        The volatility of the underlying asset.
    direction : str
        The direction of the option ("call" or "put").
    greek : str
        The greek to estimate ("delta", "gamma", "theta", or "vega").
    estimation_method : str, optional
        The method to use to estimate the option price.
    path_seed : array-like, optional
        The seed to use for the random number generator in the simulation. Default is False, else an array of size 'run' needs to be provided.
    n : int, optional
        The number of samples to use in the simulation. Default is 10000
    run : int, optional
        The number of estimates on which to compute the MSE. Default is 500
    verbose : bool, optional
        A flag indicating whether to print intermediate results during the optimization. Default is False.
        
    Returns
    -------
    float
        The optimal value of h0 that minimizes the bias of the option price estimate.
    """
    
    # Define the inequality constraint that h must be positive
    cons = {'type': 'ineq', 'fun': lambda x: x}
    
    # Minimize the bias of the option price estimate using the COBYLA method
    result = minimize(bias, h0, args=(S, K, T, r, q, sigma,direction,greek, estimation_method,path_seed, n, run, verbose ),
                      constraints=cons, method="COBYLA")
    
    # Return the optimal value of h
    return result.x

In [ ]:
optimizer(0.01 ,S, K, T, r, q, sigma, direction="call", greek="vega", 
estimation_method = "central_difference", path_seed=False, n =10000, run =500, verbose= True)

The current value of h is  [0.01]  the error is:  [0.02256334]
The current value of h is  [1.01]  the error is:  [0.08383257]
The current value of h is  [-0.115]  the error is:  [0.00022518]
The current value of h is  [-0.005625]  the error is:  [0.07003589]
The current value of h is  [0.04125]  the error is:  [0.00135722]
The current value of h is  [0.0725]  the error is:  [0.0004606]
The current value of h is  [0.088125]  the error is:  [0.00031934]
The current value of h is  [0.10375]  the error is:  [0.00023953]
The current value of h is  [0.119375]  the error is:  [0.00019794]
The current value of h is  [0.135]  the error is:  [0.00018962]
The current value of h is  [0.150625]  the error is:  [0.00015437]
The current value of h is  [0.16625]  the error is:  [0.00013519]
The current value of h is  [0.181875]  the error is:  [0.00012766]
The current value of h is  [0.1975]  the error is:  [0.00010532]
The current value of h is  [0.213125]  the error is:  [9.65539017e-05]
The current

array(0.212925)